In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Library
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np
from pydub import AudioSegment
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import IPython
import sklearn
import seaborn as sns
from sklearn.utils import shuffle
from tqdm import tqdm
import IPython.display as ipd

%load_ext tensorboard

In [ ]:
#datasets chainsaw and crackling fire
PATH_CHAINSAW = "../input/environmental-sound-classification-50/audio/audio/16000/"
CSV_CHAINSAW = "../input/environmental-sound-classification-50/esc50.csv"

#datasets gun shot
CSV_GUNSHOT = "../input/urbansound8k/UrbanSound8K.csv" 

In [ ]:
#read csv
df_chainsaw = pd.read_csv(CSV_CHAINSAW)
df_gunshot = pd.read_csv(CSV_GUNSHOT)

print(df_chainsaw.head())
print(df_gunshot.head())

In [ ]:
chainsaw = df_chainsaw.loc[df_chainsaw['category'].isin(['chainsaw', 'crackling_fire'])]
gunshot = df_gunshot[df_gunshot['class'] == 'gun_shot']

chainsaw = chainsaw.drop(['esc10', 'src_file', 'take'], axis=1)
gunshot = gunshot.drop(['fsID', 'start', 'end', 'classID', 'salience'], axis=1)
print(chainsaw)
print(gunshot)

In [ ]:
#rename the columns
gunshot = gunshot.rename(columns={'class': 'category', 'slice_file_name': 'filename'})
gunshot

In [ ]:
#combined chainsaw and gunshot datasets
combined_datasets = pd.concat([chainsaw, gunshot])
combined_datasets

In [ ]:
classes = combined_datasets['category'].unique()
print(classes)

class_dict = {i:x for x,i in enumerate(classes)}
# class_dict

combined_datasets['target'] = combined_datasets['category'].map(class_dict)
# combined_datasets

In [ ]:
sample_df = combined_datasets.drop_duplicates(subset=['target'])
sample_df

In [ ]:
#this cell just to practice of iterrows function
# for index, row in sample_df.iterrows():
#     print(type(row[0]))



In [ ]:
signals = {}
mel_spectrograms = {}
mfccs = {}

for index, row in sample_df.iterrows():
    if row['category'] == 'gun_shot':
        PATH_GUNSHOT = '../input/urbansound8k/fold' + str(row[1]) + '/' + row[0]
        signal2, rate2 = librosa.load(PATH_GUNSHOT, res_type = 'kaiser_fast') #signal2 % rate2 indicated for gunshot
        signals[row[3]] = signal2
        mel_spec = librosa.feature.melspectrogram(y=signal2 , sr=rate2, n_fft=2048, hop_length=512)
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        mel_spectrograms[row[3]] = mel_spec
        mfcc = librosa.feature.mfcc(signal2 , rate2 , n_mfcc=13, dct_type=3)
        mfccs[row[3]] = mfcc
    else:
        signal1, rate1 = librosa.load(PATH_CHAINSAW + row[0])
        signals[row[3]] = signal1
        mel_spec = librosa.feature.melspectrogram(y=signal1, sr=rate1, n_fft=2048, hop_length=512)
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        mel_spectrograms[row[3]] = mel_spec
        mfcc = librosa.feature.mfcc(signal1, rate1, n_mfcc=13, dct_type=3)
        mfccs[row[3]] = mfcc

In [ ]:
def plot_signal(signal):
    """
    this function will take the signal dictionary and plot the signals
    """
    fig , axes = plt.subplots(nrows=2, ncols=2 , sharex =False ,sharey=True,
                             figsize=(40,20))
    fig.suptitle('Time series',size=50)
    i=0
    for x in range(5):
        for y in range(2):
            axes[x,y].set_title(list(signal.keys())[i])
            axes[x,y].plot(list(signal.values())[i])
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1
            
            
def dis_feature(mfccs, cmap=None):
    """
    this function will take the mfcc/mel_spectrogram dictionary and plot the signals
    """
    fig ,axes= plt.subplots(nrows=2 , ncols=2 , sharex=False, sharey=True , figsize=(40,20))
    fig.suptitle('mel', size=50)
    i=0
    for x in range(5):
        for y in range(2):
            axes[x,y].set_title(list(mfccs.keys())[i])
            axes[x,y].imshow(list(mfccs.values())[i], cmap=cmap,interpolation='nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i+=1

In [ ]:
plot_signal(signals)

In [ ]:
dis_feature(mel_spectrograms)

In [ ]:
dis_feature(mfccs, cmap='hot')

In [ ]:
audio = AudioSegment.from_file('../input/urbansound8k/fold2/100652-3-0-0.wav', format='WAV')
audio2, samplerate = librosa.load('../input/urbansound8k/fold2/100652-3-0-0.wav')

print(len(np.array(audio.get_array_of_samples())))
print(len(audio2))

In [ ]:
print(sample_df)
for index, a in sample_df.iterrows():
    print(type(a))

In [ ]:
k, n = librosa.load('../input/urbansound8k/fold2/100652-3-0-0.wav')
j = librosa.get_duration(k, sr=n)
if librosa.get_duration(k) == 3.0:
    print("harisno")
# silence = AudioSegment.silent(duration=pad_ms-len(audio))
# padded = audio + silence  # Adding silence after the audio
# padded.export('padded-file.wav', format='wav')

In [ ]:
combined_datasets

In [ ]:
X , y = [] , []
for index, data in tqdm(combined_datasets.iterrows()):
    if data['category'] == 'gun_shot':
        #padding
        #di sini aku padding karena beberapa dataset itu ada yg di bawah 2 detik, sedangkan aku butuh narik sample dari suatu audio 2 detik secara acak, jadi aku pake silent untuk nambahin durasi suatu audio hingga 5 detik
        pad_ms = 5000 #ini durasi yg kita inginkan
        PATH_GUNSHOT = '../input/urbansound8k/fold' + str(data[1]) + '/' + data[0] #ini directory file datasets (perlu diinget di sini menggunakan iterrows, jadi jangan lupa ada index sama series)
        signal2, rate2 = librosa.load(PATH_GUNSHOT, res_type = 'kaiser_fast') #load data, menggunakan kaiser fast biar lebih cepat ekstraknya
        if librosa.get_duration(signal2, sr=rate2) < pad_ms/1000.0: #nah buat fungsi di mana jika ada audio di bawah 5 detik, dia akan dipadding
            audio = AudioSegment.from_wav('../input/urbansound8k/fold' + str(data[1]) + '/' + data[0]) #awalnya pake PATH_GUNSHOT tapi gatau kenapa ga ke-call, jadinya ditulis ulang
            silence = AudioSegment.silent(duration=pad_ms-len(audio)) #ini silence alias tambahan durasi agar 5 detik
            padded = audio + silence #ini hasil padding
            new_file = 'padded-file'+ str(index) + '.wav' #ini nama file padding yang akan diexport (ditaruh di output)
            padded.export(new_file, format='wav') #exported
            #./padded-file.wav
            signal22, rate22 = librosa.load('./'+ new_file, res_type='kaiser_fast') #load, tapi khusus file padded
            for i in range(3): #ngambil 3 sample suara selama 2 detik pada 1 audio
                n = np.random.randint(0, len(signal22)-(rate22*2)) #dikurang rate*2 berarti dikurang 2 detik
                sig_22 = signal22[n: int(n+(rate22*2))] #ditambah rate*2 artinya durasi ditambah 2 detik
                mfcc_22 = librosa.feature.mfcc(sig_22, sr=rate22, n_mfcc=13) #just another thing
                X.append(mfcc_22) 
                y.append(data[2])
        else:
            for i in range (3):
                n = np.random.randint(0, len(signal2)-(rate2*2)) #frekuensi = banyaknya cuplikan yg terjadi dalam 1 detik
                sig_2 = signal2[n: int(n+(rate2*2))]
                mfcc_2 = librosa.feature.mfcc(sig_2, sr=rate2, n_mfcc=13)
                X.append(mfcc_2)
                y.append(data[2])
    else: #untuk category non-gunshot
        signal1, rate1 = librosa.load(PATH_CHAINSAW + data[0])
        for i in range(3):
            n = np.random.randint(0, len(signal1) - (rate1*2))
            sig_1 = signal1[n: int(n+rate1*2)]
            mfcc_1 = librosa.feature.mfcc(sig_1, sr=rate1, n_mfcc= 13)
            X.append(mfcc_1)
            y.append(data[2])

In [ ]:
#THIS CELL JUST ONLY ONCE TO RUN!!!! REMEMBER, ONLY ONCE!!!!
X = np.array(X)
y = np.array(y)

y = tf.keras.utils.to_categorical(y, num_classes=3) #jan sampe fungsi ini di-run lebih dari sekali. RUN ULANG DARI AWAL JIKA FUNGSI INI RUNNED MORE THAN ONCE
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)

X.shape
y.shape 

In [ ]:
x_train , x_val , y_train , y_val = train_test_split(X , y ,test_size=0.2, random_state=2020)
INPUT_SHAPE = (13, 87, 1)

In [ ]:
# MODEL DARI 85% VALIDATION
model =  models.Sequential([
                          layers.Conv2D(16 , (3,3),activation = 'relu',padding='valid', input_shape = INPUT_SHAPE),
                          layers.Conv2D(16, (3,3), activation='relu',padding='valid'),

                          layers.Conv2D(32, (3,3), activation='relu',padding='valid'),
                          layers.Conv2D(32, (3,3), activation='relu',padding='valid'),

                          layers.Conv2D(64, (3,3), activation='relu',padding='valid'),
                          layers.Conv2D(32, (3,3), activation='relu',padding='valid'),
                          layers.GlobalAveragePooling2D(),


                          layers.Dense(32 , activation = 'relu'),
                          layers.Dense(3 , activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'acc')

In [ ]:
# #MODEL NICO
# model =  models.Sequential([
#     layers.Conv2D(64 , (3,3),activation = 'relu',padding='same', input_shape = INPUT_SHAPE),
#     layers.BatchNormalization(),
#     layers.Conv2D(64, (3,3), activation='relu',padding='same'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D((2,2), strides=(2,2)),
#     layers.Dropout(0.2),

#     layers.Conv2D(128, (3,3), activation='relu',padding='same'),                      
#     layers.BatchNormalization(),
#     layers.Conv2D(128, (3,3), activation='relu',padding='same'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D((2,2), strides=(2,2)),
#     layers.Dropout(0.2),

#     layers.Conv2D(256, (3,3), activation='relu',padding='same'),
#     layers.BatchNormalization(),
#     layers.Conv2D(256, (3,3), activation='relu',padding='same'),
#     layers.BatchNormalization(),
#     layers.MaxPooling2D((2,2), strides=(2,2)),    
#     layers.Dropout(0.2),

#     layers.GlobalAveragePooling2D(),

#     layers.Dense(256 , activation = 'relu'),
#     layers.Dense(256 , activation = 'relu'),
#     layers.Dense(len(classes) , activation = 'softmax')
# ])

# model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['acc'])

In [ ]:
model.save('path/to/location')

In [ ]:
model.summary(./padded-file7382.wav)

In [ ]:
%mkdir "cpkt"
%mkdir "logs"

In [ ]:
LOGDIR = "logs"
CPKT = "cpkt/"

In [ ]:
#this callback is used to prevent overfitting.
callback_1 = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto',
    baseline=None, restore_best_weights=False
)

#this checkpoint saves the best weights of model at every epoch
callback_2 = tf.keras.callbacks.ModelCheckpoint(
    CPKT, monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch', options=None
)

#this is for tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOGDIR)

In [ ]:
history = model.fit(x_train,y_train ,
            validation_data=(x_val,y_val),
            epochs=100,
            callbacks = [callback_1 , callback_2 , tensorboard_callback])

In [ ]:
#testing audio
sig , sr = librosa.load('../input/chainsaw-testing/chainsaw-01.wav', sr=16000)
ipd.display(ipd.Audio(sig, rate=sr))
librosa.display.waveplot(y = sig, sr = sr)

In [ ]:
class conf:
    sr = 16000
    duration = 3
    hop_length = 340*duration
    fmin = 20
    fmax = sr // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sr * duration
    epochs = 30
    
new_model = models.load_model('../input/chainsaw-testing/')

In [ ]:
def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf.sr,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

def split_audio(audio_data, w, h, threshold_level, tolerence=10):
    split_map = []
    start = 0
    data = np.abs(audio_data)
    threshold = threshold_level*np.mean(data[:25000])
    inside_sound = False
    near = 0
    for i in range(0,len(data)-w, h):
        win_mean = np.mean(data[i:i+w])
        if(win_mean>threshold and not(inside_sound)):
            inside_sound = True
            start = i
        if(win_mean<=threshold and inside_sound and near>tolerence):
            inside_sound = False
            near = 0
            split_map.append([start, i])
        if(inside_sound and win_mean<=threshold):
            near += 1
    return split_map

In [ ]:
sound_clips = split_audio(sig, 10000, 2500, 15, 10)
duration = len(sig)
i = 1

for intvl in sound_clips:
    clip, index = librosa.effects.trim(sig[intvl[0]:intvl[1]],       
                                       top_db=20, frame_length=512, hop_length=64)
    mel_spec = audio_to_melspectrogram(conf, clip)
    testing = np.array(mel_spec)
    testing = testing.reshape(1, testing.shape[0], testing.shape[1], 1)
    pred = new_model.predict(testing)
    
    blank = np.zeros(intvl[0]-0)
    blank2 = np.zeros(duration-intvl[1])
    temp = np.append(blank,clip)
    temp = np.append(temp,blank2)
    librosa.display.waveplot(y = temp, sr = sr, )
    
    print("Clip Number :", i)
    print("Interval from : ", intvl[0]/16000, " to ",intvl[1]/16000, "seconds")
    i += 1
    if(pred.max() > 0.8):
        print("Results : ", classes[np.argmax(pred)-1], "\n")
    else:
        print("Results : Unknown")
        print("Confidence Level : ", pred)
        print("Highest Confidence Level : ", classes[np.argmax(pred)-1], " of ", np.max(pred)*100, "%\n")
        ipd.display(ipd.Audio(clip, rate=sr))